In [1]:
# import the necessary packages
from keras.layers import Concatenate, Dense, Input
from tensorflow.keras.layers import Conv2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Activation, LayerNormalization
from tensorflow.keras.layers import Normalization
from tensorflow.python.keras import regularizers
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer, Lambda
# import keras.backend as K
from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import pandas as pd
# import time
# import sys
# from sklearn.model_selection import train_test_split
# import gc
# import glob
# import os, datetime
# import matplotlib.pyplot as plt
# from tensorflow import keras
import pickle as pkl
import math
from sklearn.metrics import mean_squared_error,mean_absolute_error
# from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.preprocessing import MinMaxScaler
# import numpy.linalg as la

# Loading Dataset files

In [2]:
df_Train = pd.read_csv('G:\Linux_files_packup\Paper3_Datasets\Chengdu Datasets\df_Train2scaled3.csv', nrows= 500000)

df_Test  = pd.read_csv('G:\Linux_files_packup\Paper3_Datasets\Chengdu Datasets\df_Test2scaled3.csv', nrows= 100000)

In [3]:
# df_Train.info()
# df_Train.head()

In [4]:
# df_Train['Start_hour2'].unique()

In [5]:
# df_Test['Start_hour2'].unique()

In [6]:
# df_Train.columns

In [7]:
# df_Train.drop(['Start_day2', 'Start_hour2'], axis=1, inplace=True)
# df_Test.drop(['Start_day2', 'Start_hour2'], axis=1, inplace=True)

In [8]:
# df_Train.shape

# Dividing the train and test datasets into spatial, temporal, and external feature vectors

In [9]:
def pre_process(df,h):
    df_new = df[df['Start_hour2']==h]
    
    ST_features= ['pick_Latitude', 'pick_Longitude', 'drop_Latitude','drop_Longitude', 'N_hops', 'Day', 'P_Hour',
    'haversine_distance','speed', 'week_number_of_year', 'dayofyear', 'pickup_time','pickup_cluster', 
    'dropoff_cluster', 'Heat_score_cluster','Heat_score_grid', 'PUlocation_Grid3', 'DOlocation_Grid3',
    'dayofweek_0', 'dayofweek_1', 'dayofweek_2','dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6',
    'Start_hour','Start_day', 'pick_local','pick_neighbors1', 'pick_neighbors2', 'pick_neighbors3',
    'pick_neighbors4','pick_neighbors5','pick_neighbors6','pick_neighbors7', 'pick_neighbors8']
    
    
    External = ['car_id', 'weekends_0', 'weekends_1', 'Holiday_0', 'Holiday_1','Temp', 'Weather', 'Wind',
            'Humidity', 'Visibility']
    Target = ['Trip_duration']

    ##XTrain
    df_XTrain_ST       = df_new.iloc[:,:][ST_features]
    df_XTrain_External      = df_new.iloc[:,:][External]
    y_train1  =  df_new.iloc[:,:][Target]
    
    df_XTrain_ST       = df_XTrain_ST.to_numpy()
    df_XTrain_External      = df_XTrain_External.to_numpy()
#     y_train1 =  y_train1.to_numpy()
    
    ####Train:
    ####### ST
    sample_size = df_XTrain_ST.shape[0] # number of samples in train set
    time_steps  = df_XTrain_ST.shape[1] # number of features in train set
    input_dimension = 1               # each feature is represented by 1 number

    train_ST_emb_reshaped = df_XTrain_ST.reshape(sample_size,time_steps,input_dimension)
    

    return train_ST_emb_reshaped, df_XTrain_External, y_train1

# Scaled Dot-Product Attention

In [10]:
class ScaledDotProductAttention():
    
    def __init__(self, attn_dropout=0.0):
        self.dropout = Dropout(attn_dropout)
        self.activation = Activation('selu')
        
        
    def __call__(self, q, k, v, mask):
        
        qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention = qk / tf.math.sqrt(dk)

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)  

        weights = self.activation(scaled_attention)
        output = tf.matmul(weights, v)

        return output, weights

# Build Graph Model with Tensorflow

# Import graph adjacency matrices

In [11]:
####Adj_distance
with open(r'G:\Linux_files_packup\Paper3_Datasets\Chengdu_adj_matrices\New Semantic Graph\Adj_distance_scaled.npy', 'rb') as f:
    Adj_distance = np.load(f)
    
    
####Adj_Speed
with open(r'G:\Linux_files_packup\Paper3_Datasets\Chengdu_adj_matrices\New Semantic Graph\Adj_speed_scaled.npy', 'rb') as f:
    Adj_Speed = np.load(f)
    

####Adj_Flow
with open(r'G:\Linux_files_packup\Paper3_Datasets\Chengdu_adj_matrices\New Semantic Graph\Adj_Flow_scaled.npy', 'rb') as f:
    Adj_Flow = np.load(f)

In [12]:
# Adj_distance.shape

In [13]:
# type(Adj_distance[0][0][0])

In [14]:
import scipy.sparse as sp
def normalized_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    normalized_adj = normalized_adj.astype(np.float64)
    return normalized_adj
    
def sparse_to_tuple(mx):
    mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    L = tf.SparseTensor(coords, mx.data, mx.shape)
    return tf.sparse_reorder(L) 
    
def calculate_laplacian(adj, lambda_max=1):  
    adj = normalized_adj(adj + sp.eye(adj.shape[0]))
    adj = sp.csr_matrix(adj)
    adj = adj.astype(np.float64)
    return sparse_to_tuple(adj)

In [15]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

# Graph block For Global correlations (Adj_graph matrices)

In [16]:
def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
#     print(init_range)
    initial = tf.random_uniform([batch_size,input_dim, output_dim], minval=-init_range,
                            maxval=init_range, dtype=tf.float32)

    return tf.Variable(initial,name=name)#, init_range

In [17]:
class G_GCN(object):     
    def __init__(self, num_units, adj, inputs, output_dim,dim,weightsx,weightsx2
                 ,activationx = tf.nn.relu, input_size = None, num_proj=None, reuse = True, **kwargs):
        super(G_GCN, self).__init__(**kwargs)
        if input_size is not None:
            logging.warn("%s: The input_size parameter is deprecated.", self)
        self._num_units = num_units
        self._output_dim = output_dim
        self._inputs = inputs
        self._num_nodes = adj.shape[0]
        self._input_dim = dim
        self._batch_size = tf.shape(inputs)[0]
        self._adj = adj  
        self.weightsx = weightsx
        self._activation = activationx
        self.weightsx1 = weightsx2

        
    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    @property
    def output_size(self):
        output_size = self._num_units
        return output_size
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
           
    def _gconv(self,scope=None):
        ####[batch, num_nodes, seq]
        inputs = self._inputs       
#         inputs = tf.transpose(inputs) #, perm=[2,0,1])
#        print('0',inputs.get_shape())
        x0 = inputs #tf.reshape(inputs,shape=[self._num_nodes,self._batch_size*self._input_dim])
#         print("X0:",x0)
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope, reuse=True):
            ####hidden1
            for adj in self._adj:
                adjx = tf.convert_to_tensor(adj,dtype=tf.dtypes.float32)
                x1 = adjx*x0[...,None]
#             print(x1)
#             x1 = tf.reshape(x1,shape=[self._num_nodes,self._batch_size,self._input_dim])
#             x1 = tf.reshape(x1,shape=[self._num_nodes*self._batch_size,self._input_dim])
            
#             weights = weight_variable_glorot(self._input_dim, self.output_size, name='weights')
#             print("x1:",x1)
#             weightse = tf.get_variable(
#                 'weightse', [self._input_dim, 135,135], initializer = tf.truncated_normal_initializer(stddev=0.1), dtype= tf.float64)
#             print("weightse 1:",weightse)
#             self.hidden = self._activation(tf.matmul(x1,weightse))
            self.output = self._activation(x1 * self.weightsx)
#             self.output = self._activation(tf.matmul(x1,self.weightsx) + self.bias)
#             print(self.output)
#             self.output2 = self._activation(self.output * self.weightsx1)
#             self.output2 = self._activation(tf.matmul(self.output,self.weightsx1) + self.bias1)
#             print("self.output:",self.output)
        return self.output, self.weightsx,self.weightsx1
    def _get(self):
        z2, w1,w2 = self._gconv(self)
#         with tf.Session() as sess:  print(w.eval())
        return z2, w1,w2    

In [18]:
def Build_Model(train_ST_emb_reshaped,train_Ex_emb,Adj_distance1,Adj_Speed1,Adj_Flow1,weights_G1,weights1_G1):

    numerics  = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'uint8']
    input_ST0 = tf.convert_to_tensor(train_ST_emb_reshaped, dtype=tf.dtypes.float32)
    input_XT0 = tf.convert_to_tensor(train_Ex_emb,dtype=tf.dtypes.float32)
    
    Adj_distance2 = Adj_distance1
    Adj_Speed2 = Adj_Speed1
    Adj_Flow2  = Adj_Flow1
#     print(f'Adj_distance2 is {Adj_distance2[0][0:5]}')
    
    weights_G2 = weights_G1
    weights2_G2= weights1_G1

    
    ###### Spatial-Temporal Features module:    
    ### TCN Block1 for Spatial-Temporal
#     input_shape1 = tf.keras.Input(shape=(train_ST_emb_reshaped.shape[1],train_ST_emb_reshaped.shape[2]))
    TCN1 = tf.keras.layers.Conv1D(filters=2, kernel_size=5, strides=1, dilation_rate=1, padding='causal', 
                                    name='Conv1D_TCN1')(input_ST0)
    TCN1 = tf.keras.activations.relu(TCN1)
    TCN1 = tf.keras.layers.Dropout(0.0)(TCN1)
    TCN1 = tf.keras.layers.MaxPooling1D(pool_size=4,strides=1)(TCN1)
#     print("The TCN1 output is ", TCN1.shape)
    
    ### TCN Block2 for Spatial-Local
    TCN2 = tf.keras.layers.Conv1D(filters=2, kernel_size=5, strides=1, dilation_rate=2, padding='causal', 
                                     name='Conv1D_TCN2')(input_ST0)
    TCN2 = tf.keras.activations.relu(TCN2)
    TCN2 = tf.keras.layers.Dropout(0.0)(TCN2)
    TCN2 = tf.keras.layers.AveragePooling1D(pool_size=4,strides=1)(TCN2)
#     print("The TCN2 output is ", TCN2.shape)
    
    ### TCN Block3 for combination features
#     combination = Concatenate(axis=1)([input_shape1, input_shape2])
    TCN3 = tf.keras.layers.Conv1D(filters=2, kernel_size=5, strides=1, dilation_rate=4, padding='causal', 
                                     name='Conv1D_TCN3')(input_ST0)
    TCN3 = tf.keras.activations.relu(TCN3)
    TCN3 = tf.keras.layers.Dropout(0.0)(TCN3)
    TCN3 = tf.keras.layers.MaxPooling1D(pool_size=4,strides=1)(TCN3)
#     print("The TCN3 output is ", TCN3.shape)
    
    
    #### Self-attention module for Spatio-temporal correlations
    TCN_Mul1  = tf.keras.layers.Multiply()([ TCN1, TCN2])
    TCN_Mul2  = tf.keras.layers.Multiply()([ TCN_Mul1, TCN3])
#     print("The Spatial-Temporal Features module output is ", TCN_Mul2.shape)
    TCN_Mul2 = tf.keras.layers.Flatten()(TCN_Mul2)
#     print("The Spatial-Temporal Features module output is ", TCN_Mul2.shape)
    
    
    ##### Spatial-Temporal Graphs module:    
    ### Speed Graph Block
    GGCN_STS = G_GCN(135, Adj_Speed2, TCN_Mul2, 126, 66,weights_G2,weights2_G2)
    GGCN_STS,w1,w2 = GGCN_STS._get()
#     print("GGCN_STS:", GGCN_STS)
    
    
    ### Distance Graph Block
    GGCN_STD = G_GCN(135, Adj_distance2, TCN_Mul2, 126, 66,weights_G2,weights2_G2)
    GGCN_STD,w1,w2 = GGCN_STD._get()
#     print("GGCN_STD:", GGCN_STD)
    
    
    ### Flow Graph Block
    GGCN_STF = G_GCN(135, Adj_Flow2, TCN_Mul2, 126, 66,weights_G2,weights2_G2)
    GGCN_STF,w1,w2 = GGCN_STF._get()
#     print("GGCN_STF:", GGCN_STF)
    
    
    
    #### Weight sharing layers for calculate similarity
    Weight_layer1 = Lambda(function=lambda x: 1 - K.abs(x[0] - x[1]), output_shape=lambda x: x[0],
                    name='L1_distance1')([GGCN_STS, GGCN_STD])
    Weight_layer2 = Lambda(function=lambda x: 1 - K.abs(x[0] - x[1]), output_shape=lambda x: x[0],
                    name='L1_distance2')([GGCN_STD, GGCN_STF])
    Weight_layer2 = tf.transpose(Weight_layer2, perm=[0,2,1])
    
#     print("Weight_layer1:", Weight_layer1)
#     print("Weight_layer2:", Weight_layer2)
    
    
    
    
     ####TCN_Cross Gated module for Graph Fusion representation:
    Cross_Mul  = tf.matmul(Weight_layer1,Weight_layer2)
#     Cross_Mul  = tf.keras.layers.Multiply()([ Weight_layer1, Weight_layer2])
# #     Cross_Mul  = tf.transpose(Cross_Mul, perm=[0,2,1])
# #     Cross_Mul = tf.reshape(Cross_Mul, shape=[Cross_Mul.shape[0], Cross_Mul.shape[1],1])
#     print("Cross_Mul:", Cross_Mul)
#     Cross_Mul2 = tf.keras.layers.Flatten()(Cross_Mul)
#     print("Cross_Mul2:", Cross_Mul2)

    TCN4 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, dilation_rate=3, padding='causal', 
                                    name='Conv1D_TCN4')(Cross_Mul)
    TCN4 = tf.keras.activations.relu(TCN4)
    TCN4 = tf.keras.layers.Dropout(0.0)(TCN4)
    TCN4 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1)(TCN4)
#     print("The TCN4 output is ", TCN4.shape)
    
    TCN5 = tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, dilation_rate=2, padding='causal', 
                                    name='Conv1D_TCN5')(Cross_Mul)
    TCN5 = tf.keras.activations.relu(TCN5)
    TCN5 = tf.keras.layers.Dropout(0.0)(TCN5)
    TCN5 = tf.keras.layers.AveragePooling1D(pool_size=2,strides=1)(TCN5)
#     print("The TCN5 output is ", TCN5.shape)
    
#     Graph_out = Lambda(function=lambda x: 1 - K.abs(x[0] - x[1]), output_shape=lambda x: x[0],
#                     name='L1_distance1')([TCN4, TCN5])
    
    
    Graph_out  = Concatenate(axis=1)([TCN4, TCN5])
#     Graph_out  = tf.keras.layers.Multiply()([ TCN4, TCN5])
#     print("The Spatial-Temporal Graph module output is ", Graph_out.shape)
    Graph_out = tf.keras.layers.Flatten()(Graph_out)
#     print("The Spatial-Temporal Graph module output after flatten is ", Graph_out.shape)
    

    ####DualGated ResNet module for External features:
#     input_shape2 = tf.keras.Input(shape=(train_Ex_emb.shape[1],))
    Ext_layer1 = Dense(64,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(input_XT0)
    Ext_layer2 = Dense(64,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(input_XT0)
    Block1_out1= tf.keras.layers.Multiply()([Ext_layer1, Ext_layer2])
#     BN_Layer   = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(Block1_out1,training=True)
    Ext_layer3 = Dense(128,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(Block1_out1)
    Ext_layer4 = Dense(128,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(Block1_out1)
    Block1_out2= Concatenate(axis=1)([Ext_layer3, Ext_layer4])
    Block1_out2 = tf.keras.layers.Flatten()(Block1_out2)


    ######Gated Fusion Module for ALL Features representation 
    Stage1 = Dense(256,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(Graph_out)
#     Stage1_flatten = tf.keras.layers.Flatten()(Stage1)
#     Stage2_flatten = tf.keras.layers.Flatten()(Block1_out2)
    Stage2 = Dense(128,use_bias=False,kernel_initializer="he_uniform", activation= 'relu',
               kernel_regularizer= regularizers.l2(0.001))(Block1_out2)
    Geted_concat = Concatenate(axis=1)([Stage1, Stage2])
#     print(f"Geted_concat output shape is {Geted_concat.shape}")
    
    
#     #### Transformer MHA Module
#     self_hidden_size = Geted_concat.shape[1]
#     Final_MHA_Block = InterpretableMultiHeadAttention(num_heads, self_hidden_size, dropout=0.0)
#     Final_MHA_Block_output, Final_MHA_self_att  = Final_MHA_Block(Geted_concat, Geted_concat, Geted_concat, mask=None)
#     print(f"Final_MHA_Block_output shape is {Final_MHA_Block_output.shape}")
    
    

    #####FC layers and Dense for output######
    # Layer 1
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(Geted_concat,training=True)
    FC = Dense(128,use_bias=False,kernel_initializer="he_uniform", name = "FC1",
               kernel_regularizer= regularizers.l2(0.01))(Geted_concat)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Activation('relu')(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dropout(0.1)(FC)
    
    # Layer 2
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dense(256,use_bias=False,kernel_initializer="he_uniform", name = "FC2",
               kernel_regularizer= regularizers.l2(0.01))(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Activation('relu')(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dropout(0.1)(FC)
    

    # Layer 3
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dense(128,use_bias=False,kernel_initializer="he_uniform",name = "FC3",
               kernel_regularizer= regularizers.l2(0.01))(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Activation('relu')(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dropout(0.1)(FC)

    # Layer 4
#     model = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dense(64,use_bias=False,kernel_initializer="he_uniform", name = "FC4",
               kernel_regularizer= regularizers.l2(0.01))(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Activation('relu')(FC)
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dropout(0.1)(FC)
    
        # Layer 5
#     FC = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    FC = Dense(32,use_bias=False,kernel_initializer="he_uniform", name = "FC5", 
               kernel_regularizer= regularizers.l2(0.01))(FC)
#     FC = Dropout(0.1)(FC)
    # model = BatchNormalization(epsilon=1e-06, momentum=0.99, weights=None, trainable=False)(model,training=True)
    FC = Activation('relu')(FC)

    
    
    #Linear Dense Output layer
#     mlp_out = LayerNormalization(epsilon=1e-06, weights=None, trainable=False)(FC,training=True)
    output = Dense(1,activation='linear')(FC)
    
    
#     output = tf.reshape(output, shape=[-1])
#     _weights = tf.reshape(_weights, shape=[-1])
#     output = output*_weights + _biases
#     output = tf.reshape(output, shape=[-1,1])
#     output = tf.cast(output, dtype='float32')
    
    
    return output, w1, w2

# Train and Test model for each hour

In [19]:
####Delete all flags before declare#####
# batch_size = 16

# for name in list(flags.FLAGS):
#       delattr(flags.FLAGS,name)


tf.app.flags.DEFINE_string('f', '', 'kernel')
###### Settings ######
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('training_epoch', 30, 'Number of epochs to train.')
flags.DEFINE_integer('batch_size', 32, 'batch size.')
batch_size = FLAGS.batch_size
lr = FLAGS.learning_rate
training_epoch = FLAGS.training_epoch
num_nodes = 135
# batch_size = 16

labels = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# ADJ = tf.placeholder(shape=[135, 135], dtype=tf.float64)

inputs1 = tf.placeholder(shape=[None, 36,1], dtype=tf.float32)
inputs2 = tf.placeholder(shape=[None, 10], dtype=tf.float32)
inputs  = [inputs1, inputs2]


weights = weight_variable_glorot(66,126, name='weights')
weights1 = weight_variable_glorot(126,126, name='weights1')


# weights = {
#     'out': tf.Variable(tf.random_normal([64, 1], mean=1.0),dtype=tf.float32, name='weight_o')}
# biases = {
#     'out': tf.Variable(tf.random_normal([1]),dtype=tf.float32,name='bias_o')}

##First hour (define i as = 5) 
i = 5 
# hr = tf.convert_to_tensor(10, dtype=tf.int32)
# dftrain1 = train_ST_emb_reshaped[0:batch_size]
# dftrain2 = train_Ex_emb[0:batch_size]

# train_ST_emb_reshaped,train_Ex_emb
# pred, w1,b1, w2,b2 = Build_Model(df1, 
#                Adj_distance[i],Adj_Speed[i],Adj_Flow[i], weights_ST, bias_ST, weights_ST1, bias_ST1,weights ,biases)

pred,w1,w2 = Build_Model(inputs1,inputs2,Adj_distance[i],Adj_Speed[i],Adj_Flow[i],weights,weights1)
y_pred = pred

##loss
mae_loss = tf.keras.losses.MeanAbsoluteError()
loss =  tf.reduce_mean(mae_loss(y_pred,labels))
# loss = tf.reduce_mean(tf.keras.metrics.mean_absolute_error(y_pred,labels))
# loss = tf.reduce_mean(tf.square(y_pred-labels))
# loss = tf.reduce_mean(tf.squared_difference(y_pred, labels))
##rmse

# error = tf.reduce_mean(tf.abs(labels - y_pred))
error = tf.reduce_mean(tf.keras.metrics.mean_absolute_error(labels, y_pred))
# error = tf.reduce_mean(tf.square(y_pred-labels))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)


###### Initialize session ######
# variables = tf.initialize_all_variables()
variables = tf.global_variables()
saver = tf.train.Saver(tf.global_variables())  
#sess = tf.Session()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())


def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


# path = './model'
    
###### evaluation ######
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae  = mean_absolute_error(a,b)
    mape = MAPE (a,b)
#     F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
#     r2  = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
#     var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, mape

H_RMSE, H_MAE, H_MAPE = [], [], []
model_rmse, model_mae, model_mape = [], [], []
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_pred, test_map = [],[],[],[],[]
# time_start = time.time()
print(f'We start train model')
for i in range(5,23):
    train_ST_emb_reshaped, train_Ex_emb, df_trainy   = pre_process(df_Train, i)
    test_ST_emb_reshaped, test_Ex_emb, df_testy       = pre_process(df_Test, i)
    totalbatch = int(train_ST_emb_reshaped.shape[0]/batch_size)
    training_data_count = len(train_ST_emb_reshaped)
    totalbatchT = int(test_ST_emb_reshaped.shape[0]/batch_size)
    MAE1  = np.empty([totalbatchT], dtype=float)
    MAE1.fill(1000)
    RMSE1 = np.empty([totalbatchT], dtype=float)
    RMSE1.fill(1000)
    MAPE1  = np.empty([totalbatchT], dtype=float)
    MAPE1.fill(1000)
    
    print(f'We train model for hour {i}')
#     print(f'train_size= ({train_ST_emb_reshaped.shape[0]},{train_Ex_emb.shape[0]}, {df_trainy.shape[0]})')
#     print(f'test_size= ({test_ST_emb_reshaped.shape[0]},{test_Ex_emb.shape[0]}, {df_testy.shape[0]})')
    if i ==5:
        pred,w1,w2 = Build_Model(inputs1,inputs2,Adj_distance[i-5],Adj_Speed[i-5],Adj_Flow[i-5],weights,weights1)
    else:
        pred,w1,w2 = Build_Model(inputs1,inputs2,Adj_distance[i-5],Adj_Speed[i-5],Adj_Flow[i-5],w1,w2)
    for epoch in range(training_epoch):
        print(f'we start train epoch:{epoch}')
#         st_epoch = time.time()
        for m in range(totalbatch):
            mini_batch1 = train_ST_emb_reshaped[m * batch_size : (m+1) * batch_size]
            mini_label  = df_trainy[m * batch_size : (m+1) * batch_size]
            mini_batch2 = train_Ex_emb[m * batch_size : (m+1) * batch_size]
            op, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                     feed_dict = {inputs[0]:mini_batch1, inputs[1]:mini_batch2, labels:mini_label})
            batch_loss.append(loss1)
#             max_value = np.max(mini_label) 
            batch_rmse.append(rmse1)
#             et_epoch = time.time()
#             epoch_time = et_epoch - st_epoch
#         print(f'batch RMSE:{rmse1}')
#         print(f'Train_batch Loss:{loss1}')
#         op.get_weights()
#         print(f'batch ACC:{acc}')
#     print(f'w is:',WW[0][0][0:5])
#         print(f"Epoch time {epoch} is: {epoch_time} sec")

        test_lossb, test_predb,test_rmseb,test_maeb = [],[],[],[]
    
        for m in range(totalbatchT):
            mini_batchT1 = test_ST_emb_reshaped[m * batch_size : (m+1) * batch_size]
            mini_batchT2 = test_Ex_emb[m * batch_size : (m+1) * batch_size]
            mini_labelT = df_testy[m * batch_size : (m+1) * batch_size]
#         test_T.append(mini_batchT)
#         testy_T.append(mini_labelT)
#     test_T = np.array(test_T)
#     testy_T = np.array(testy_T)
            lossb, rmseb, test_outputb = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs[0]:mini_batchT1, inputs[1]:mini_batchT2,labels:mini_labelT})
            test_labelb = mini_labelT.values
#             print(f'Test_batch Loss:{lossb}')
            rmseb, maeb,mape  = evaluation(test_labelb, test_outputb)
            if maeb < MAE1[m]:
                MAE1[m] = maeb
            if rmseb < RMSE1[m]:
                RMSE1[m] = rmseb
            if mape < MAPE1[m]:
                MAPE1[m] = mape
          
      
    H_RMSE.append(RMSE1)
    H_MAE.append(MAE1)
    H_MAPE.append(MAPE1)
#         rmse = np.mean(np.array(test_rmseb))  
#         mae = np.mean(np.array(test_maeb)) 
#         acc = np.mean(np.array(test_accb)) 
#         r2_score = np.mean(np.array(test_r2b)) 
#         var_score = np.mean(np.array(test_varb)) 
#     test_predb = np.array(test_predb).ravel()
#     test_lossb = np.array(test_lossb).ravel()
#     test_label = df_newT[Target].values
#     max_value1 = np.max(df_newT[Target]) 
#     test_label1 = test_label 
#     test_output1 = test_predb 
#     test_loss.append(test_lossb)
#         test_rmse.append(rmse )
#         test_mae.append(mae)
#         test_acc.append(acc)
#         test_r2.append(r2_score)
#         test_var.append(var_score)
#     test_pred.append(test_output1)
    R,M,P =[],[],[]
    for i in range(len(H_RMSE)):
        R.append(np.mean(np.array(H_RMSE[i])))
        M.append(np.mean(np.array(H_MAE[i])))
        P.append(np.mean(np.array(H_MAPE[i])))

    model_rmse.append(np.mean(np.array(R)))
    model_mae.append(np.mean(np.array(M)))
    model_mape.append(np.mean(np.array(P)))
    print(f'Hour {i+5} RMSE:{np.mean(np.array(R))}')
    print(f'Hour {i+5} MAE:{np.mean(np.array(M))}')
    print(f'Hour {i+5} MAPE:{np.mean(np.array(P))}')
            
#     h_RMSE = np.min(test_rmse)
#     h_MAE  = np.min(test_mae)
#     print(f'RMSE for Hour is : {h_RMSE}')
#     print(f'MAE for Hour is : {h_MAE}')

        
#     H_RMSE.append(h_RMSE)
#     H_MAE.append(h_MAE)
#     h_RMSE = 0
#     h_MAE  = 0
#     h_ACC  = 0
        

        
print("Model RMSE = ", np.mean(np.array(model_rmse)))
print("Model MAE  = ", np.mean(np.array(model_mae)))
print("Model MAPE = ", np.mean(np.array(model_mape)))

We start train model
We train model for hour 5
we start train epoch:0
we start train epoch:1
we start train epoch:2
we start train epoch:3
we start train epoch:4
we start train epoch:5
we start train epoch:6
we start train epoch:7
we start train epoch:8
we start train epoch:9
we start train epoch:10
we start train epoch:11
we start train epoch:12
we start train epoch:13
we start train epoch:14
we start train epoch:15
we start train epoch:16
we start train epoch:17
we start train epoch:18
we start train epoch:19
we start train epoch:20
we start train epoch:21
we start train epoch:22
we start train epoch:23
we start train epoch:24
we start train epoch:25
we start train epoch:26
we start train epoch:27
we start train epoch:28
we start train epoch:29
Hour 5 RMSE:238.9420703433751
Hour 5 MAE:153.92284440994263
Hour 5 MAPE:21.005790374032774
We train model for hour 6
we start train epoch:0
we start train epoch:1
we start train epoch:2
we start train epoch:3
we start train epoch:4
we start tr

Hour 14 RMSE:71.09785913216203
Hour 14 MAE:43.351268576756155
Hour 14 MAPE:5.753906427139539
We train model for hour 15
we start train epoch:0
we start train epoch:1
we start train epoch:2
we start train epoch:3
we start train epoch:4
we start train epoch:5
we start train epoch:6
we start train epoch:7
we start train epoch:8
we start train epoch:9
we start train epoch:10
we start train epoch:11
we start train epoch:12
we start train epoch:13
we start train epoch:14
we start train epoch:15
we start train epoch:16
we start train epoch:17
we start train epoch:18
we start train epoch:19
we start train epoch:20
we start train epoch:21
we start train epoch:22
we start train epoch:23
we start train epoch:24
we start train epoch:25
we start train epoch:26
we start train epoch:27
we start train epoch:28
we start train epoch:29
Hour 15 RMSE:66.90066714328658
Hour 15 MAE:40.83434503334778
Hour 15 MAPE:5.409138102065118
We train model for hour 16
we start train epoch:0
we start train epoch:1
we st